In [1]:
from bs4 import BeautifulSoup
import urllib.request
import requests
import json
import time
import pandas as pd
import math

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/koushikchennakesavan/.wdm/drivers/chromedriver/mac64/100.0.4896.60/chromedriver] found in cache


In [3]:
names = []
links = []
prices = []
dates = []
mileage_list = []
transmission_list = []
engine_list = []
df_total = pd.DataFrame(columns = ['name', 'price', 'date', 'link', 'mileage', 'transmission', 'engine'])

In [4]:
driver.get("https://bringatrailer.com/ford/shelby-mustang/")
try:
    l = driver.find_element_by_xpath("//button[@data-bind='click: filter.next, with: observerItems']")
    while l.is_displayed():
        driver.execute_script("arguments[0].click();", l)
        time.sleep(1)
except: 
    page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')
c = soup.find_all('div', class_='blocks')[0]
blks = c.find_all('div', class_='block')

/var/folders/q0/d1fpvqkd2kz9d777d335jqsw0000gn/T/ipykernel_1734/2984934253.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  l = driver.find_element_by_xpath("//button[@data-bind='click: filter.next, with: observerItems']")


In [5]:
for blk in blks:
    link = blk.find('a')
    text = blk.find('div', class_='subtitle').get_text()
    name = blk.find('h3').get_text()
    mileage = ''
    transmission = ''
    engine = ''
    for i in text.split(): 
        if '$' in i:
            prices.append(i)
        elif '/' in i:
            dates.append(i)
    names.append(name)
    links.append(link['href'])
    try:
        driver.get(link['href'])
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        c = soup.find_all('li', class_='')
        print(c)
        for params in c:
            
            if 'miles' in str(params.get_text()) or 'Miles' in str(params.get_text()):
                mileage = params.get_text()
                mileage_list.append(mileage)
                print(mileage)
            if 'Transmission' in str(params.get_text()) or 'transmission' in str(params.get_text()):
                transmission = params.get_text()
                transmission_list.append(transmission)
                print(transmission)
            if 'V6' in str(params.get_text()) or 'V8' in str(params.get_text()):
                engine = params.get_text()
                engine_list.append(engine)
                print(engine)
    except:
        print("no link found")
names = pd.Series(names, name='name')
prices = pd.Series(prices, name='price')
dates = pd.Series(dates, name='date')
links = pd.Series(links, name='link')
mileage_list = pd.Series(mileage_list, name='mileage')
transmission_list = pd.Series(transmission_list, name='transmission')
engine_list = pd.Series(engine_list, name='engine')
df = pd.concat([names, prices, dates, links, mileage_list, transmission_list, engine_list], axis=1)
df_total = pd.concat([df, df_total], ignore_index=True)
print(df_total)     

[<li>Chassis: <a href="https://www.google.com/search?q=1ZVHT88S975200027" target="_blank">1ZVHT88S975200027</a></li>, <li>11k Miles</li>, <li>Supercharged 5.4-Liter V8</li>, <li>Six-Speed Manual Transmission</li>, <li>Torch Red Paint w/White Stripes</li>, <li>Black &amp; Red Leather Upholstery</li>, <li>18" Machined Aluminum Wheels</li>, <li>Brembo Front Brake Calipers</li>, <li>SVT-Tuned Suspension</li>, <li>Rear Spoiler</li>, <li>Shaker 1000 Audio System</li>, <li>Ford Certificate</li>, <li>Window Sticker</li>, <li>Clean Carfax Report</li>]
11k Miles
Supercharged 5.4-Liter V8
Six-Speed Manual Transmission
[<li>Chassis: <a href="https://www.google.com/search?q=1FA6P8JZ9J5501187" target="_blank">1FA6P8JZ9J5501187</a></li>, <li>19k Miles </li>, <li>Replacement 5.2-Liter V8</li>, <li>Six-Speed Manual Transmission</li>, <li>Limited-Slip Differential</li>, <li>Ruby Red Metallic Paint</li>, <li>Black Cobra Nogaro Seats</li>, <li>APR Performance Splitter &amp; Spoiler</li>, <li>AP Racing Fro

In [ ]:
print(df)

                                                  name    price     date  \
0              439-Mile 2016 Ford Mustang Shelby GT350  $52,000   5/4/22   
1          1,800-Mile 2009 Ford Mustang Shelby GT500KR  $55,000   5/3/22   
2    Ex–Carroll Shelby 2007 Ford Shelby GT500 40th ...  $71,500   5/2/22   
3    208-Mile 2008 Ford Mustang Shelby GT500 Super ...  $90,500   5/1/22   
4    6k-Mile 2014 Ford Mustang Shelby GT500 Super S...  $93,277   5/1/22   
..                                                 ...      ...      ...   
277           900-Mile 2017 Ford Mustang Shelby GT350R  $64,100  6/29/18   
278            6K-Mile 2016 Ford Mustang Shelby GT350R  $64,500  3/28/18   
279          1700-Mile 2016 Ford Shelby Mustang GT350R  $64,500  1/12/18   
280                    2016 Ford Mustang Shelby GT350R  $63,000  12/7/17   
281       2016 Ford Mustang Shelby GT350 Track Package  $44,250  8/31/17   

                                                  link      mileage  \
0    https://bri

In [ ]:
BaTCSV = df_total.to_csv('ford_shelby_BaT.csv', index = True)